In [1]:
import numpy as np
import pandas as pd

In [2]:
#consisit of 25M ratings
rating_df = pd.read_csv('ratings.csv')
movie_df = pd.read_csv('movies.csv')

In [3]:
print(rating_df.shape)

(25000095, 4)


In [4]:
print(movie_df.shape)

(62423, 3)


In [5]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
new_rating_df = rating_df.groupby('movieId')['rating'].count().reset_index(name='No_of_votes_v')
new_rating_df

,movieId,No_of_votes_v
0,1,57309
1,2,24228
2,3,11804
3,4,2523
4,5,11714
...,...,...
59042,209157,1
59043,209159,1
59044,209163,1
59045,209169,1


In [8]:
mean_votes_for_each_movie = rating_df.groupby('movieId')['rating'].mean().reset_index(name='Avg_Rating_R')
mean_votes_for_each_movie

,movieId,Avg_Rating_R
0,1,3.893708
1,2,3.251527
2,3,3.142028
3,4,2.853547
4,5,3.058434
...,...,...
59042,209157,1.500000
59043,209159,3.000000
59044,209163,4.500000
59045,209169,3.000000


In [9]:
new_rating_df = pd.merge(new_rating_df, mean_votes_for_each_movie, on="movieId")
new_rating_df

,movieId,No_of_votes_v,Avg_Rating_R
0,1,57309,3.893708
1,2,24228,3.251527
2,3,11804,3.142028
3,4,2523,2.853547
4,5,11714,3.058434
...,...,...,...
59042,209157,1,1.500000
59043,209159,1,3.000000
59044,209163,1,4.500000
59045,209169,1,3.000000


In [10]:
mean_vote_C = new_rating_df['Avg_Rating_R'].mean()
mean_vote_C

3.0713739209511313

In [11]:
minimum_votes_required_m = new_rating_df['No_of_votes_v'].quantile(0.86)
minimum_votes_required_m

183.0

In [12]:
# Eliminate movies with no. of votes lower than minimum_votes_required_m
new_rating_df_shortened = new_rating_df.copy().loc[new_rating_df['No_of_votes_v'] >= minimum_votes_required_m]
new_rating_df_shortened

,movieId,No_of_votes_v,Avg_Rating_R
0,1,57309,3.893708
1,2,24228,3.251527
2,3,11804,3.142028
3,4,2523,2.853547
4,5,11714,3.058434
...,...,...,...
57049,203222,258,3.149225
57517,204352,185,3.383784
57585,204542,190,3.139474
57653,204698,818,4.050733


In [13]:
new_rating_df_shortened["Weighted_Rating_W"] = (new_rating_df_shortened["Avg_Rating_R"] * new_rating_df_shortened["No_of_votes_v"] + mean_vote_C * minimum_votes_required_m) / (new_rating_df_shortened["No_of_votes_v"] + minimum_votes_required_m)
new_rating_df_shortened

,movieId,No_of_votes_v,Avg_Rating_R,Weighted_Rating_W
0,1,57309,3.893708,3.891090
1,2,24228,3.251527,3.250177
2,3,11804,3.142028,3.140949
3,4,2523,2.853547,2.868278
4,5,11714,3.058434,3.058633
...,...,...,...,...
57049,203222,258,3.149225,3.116919
57517,204352,185,3.383784,3.228428
57585,204542,190,3.139474,3.106063
57653,204698,818,4.050733,3.871690


In [14]:
# new_rating_df_shortened = pd.merge(new_rating_df_shortened,movie_df,on='movieId', how='left')
# new_rating_df_shortened

In [15]:
new_rating_df_shortened.sort_values('Weighted_Rating_W', ascending=False).head(10)

,movieId,No_of_votes_v,Avg_Rating_R,Weighted_Rating_W
314,318,81482,4.413576,4.410568
38361,159817,1747,4.464797,4.332674
840,858,52498,4.324336,4.319984
42953,171011,1124,4.483096,4.285433
49,50,55366,4.284353,4.280357
1190,1221,34188,4.261759,4.255421
522,527,60411,4.247579,4.244027
42806,170705,1356,4.398599,4.240781
1930,2019,13367,4.254769,4.238787
1173,1203,16569,4.243014,4.230215


In [16]:
# contain_values = new_rating_df_shortened[new_rating_df_shortened['genres'].str.contains('Comedy')]
# contain_values.sort_values('weightedRating', ascending=False).head(10)

In [17]:
# new_rating_df_shortened_final = pd.merge(movie_df[['movieId', 'title']], new_rating_df_shortened[['movieId', 'weightedRating']], on="movieId")
# new_rating_df_shortened_final

In [18]:
new_rating_df_shortened.to_csv("ratings_weighted.csv", index=False)